## Target Wait Time Scheduler

#### Variables
Define the decision variables:
- $X_{ijk}$: Binary variable indicating if surgery $i$ is scheduled on day $j$ in room $k$.
- $SWT_{i}$: Scheduled wait time for surgery $i$.
  - $SWT_{i} = j$ (where $X_{ijk} = 1$) + $AD_i$
- $Y_i$: Binary variable indicating whether the scheduled wait time for surgery $i$ exceeds the target wait time.

#### Parameters
Define the constants used in the model:
- $C_{jk}$: Room capacity on day $j$.
- $D_i$: Surgery $i$ duration.
- $T_i$: Target wait time for surgery $i$.
- $AD_i$: Admitted date for surgery $i$.
- $M$: A sufficiently large constant.

#### Constraints
Define the constraints:
- **Constraint 1: Surgery can only be done once**
  - $\sum_{jk} (X_{ijk}) \leq 1$ for all $i \in I$
- **Constraint 2: Room capacity**
  - $C_{jk} \geq \sum_{i \in I} (X_{ijk} \cdot D_i)$ for all $j$ in $J$, $k$ in $K$
- **Constraint 3: Set $SWT_{i}$ equal to Day Scheduled + Waited Time** 
  - $SWT_{i} \geq \sum_{jk} (j + AD_i) \cdot X_{ijk}$ for all $i$ in $I$

- **Constraint 4: Y Indicator for when $SWT_{i} < T_{i}$**
  - $T[i] \geq SWT[i] - M \cdot Y[i]$ for all $i \in I$(Activation Constraint)
  - $T[i] \leq SWT[i] + M \cdot (1 - Y[i])$ for all $i \in I$(Deactivation Constraint)

#### Objective Function
Define the objective function:
- **Minimize** $\text{Minimize} \sum_{i,j,k} \left( \text{max}\left(0, X_{ijk} \cdot (SWT_i - T_i)\right) + \beta \cdot SWT_i + \delta \cdot {penalty\_unscheduled}\right) $
- **Minimize** $\text{Minimize} \sum_{i,j,k} \left(( Y_i \cdot (SWT_i - T_i)+ \beta \cdot SWT_i + \delta \cdot {penalty\_unscheduled}\right) $






In [1]:
#Use overleaf


In [281]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

#maximize M value

def rolling_horizon_scheduler(surgical_info_df, capacity_info_df, current_day, schedule_df=None, days_ahead = 5, beta = 0.1 , delta = 99999999999):
    # Create a new model
    model = gp.Model("Rolling_Horizon_Scheduler")

    # Sets
    surgeries = surgical_info_df['id'].tolist()
    days = [current_day + i for i in range(days_ahead)]  # Days within the rolling horizon
    rooms = capacity_info_df['room'].unique().tolist()

    # Parameters
    C = {(row['day'], row['room']): row['capacity'] for _, row in capacity_info_df.iterrows()} # Capacities for each room on each day
    D = {row['id']: row['duration'] for _, row in surgical_info_df.iterrows()} # Surgery durations
    T = {row['id']: row['ideal_waiting_time'] for _, row in surgical_info_df.iterrows()} # Target wait times
    AD = {row['id']: row['days_waited'] for _, row in surgical_info_df.iterrows()} # Admitted dates
    M = 99999 # A large constant

    # Variables
    X = {(i, j, k): model.addVar(vtype=GRB.BINARY, name=f"X_{i}_{j}_{k}") for i in surgeries for j in days for k in rooms}
    SWT = {i: model.addVar(vtype=GRB.CONTINUOUS, name=f"SWT_{i}") for i in surgeries}
    Y = {i: model.addVar(vtype=GRB.BINARY, name=f"Y_{i}") for i in surgeries}
    Z = {i: model.addVar(vtype=GRB.BINARY, name=f"Z_{i}") for i in surgeries}

    # Constraints
    # Constraint 1: Surgery can only be done once
    for i in surgeries:
        model.addConstr(gp.quicksum(X[i, j, k] for j in days for k in rooms) <= 1, name=f"One Surgery{i}")

    # Constraint 2: Room capacity
    for j in days:
        for k in rooms:
            model.addConstr(gp.quicksum(X[i, j, k] * D[i] for i in surgeries) <= C[j, k], name=f"Capacity{j}_{k}")

    #Constraints for variables:
    # Constraint 3: Set SWT[i] equal to the scheduled wait time plus the admitted date
    for i in surgeries:
        model.addConstr(SWT[i] == gp.quicksum((j+ AD[i]) * X[i, j, k] for j in days for k in rooms),
                        name=f"SWT_Constraint_{i}")
    # for i in surgeries:
    #     model.addConstr(
    #         SWT[i] == gp.quicksum((j - AD[i]) * X[i, j, k] for j in days for k in rooms) + 
    #                 (current_day - AD[i]) * (1 - gp.quicksum(X[i, j, k] for j in days for k in rooms)),
    #         name=f"SWT_Constraint_{i}"
    #     )


    # Constraint 4: Ensure binary decision variable (Y[i]) respects wether SWT[i] > T[i]
    for i in surgeries:
        # model.addConstr(SWT[i] - T[i] <= M * (1 - Y[i]), name=f"Y_activate_{i}")
        # model.addConstr(SWT[i] - T[i] >= -M * Y[i], name=f"Y_deactivate_{i}")
        # #reversed?
        # model.addConstr((T[i] - SWT[i]) >= -1 * M *(1 - Y[i]), name=f"Y_deactivate_{i}")
        # model.addConstr(T[i] - SWT[i] <= M * (1 - Y[i]), name=f"Y_activate_{i}")
        # model.addConstr(SWT[i] - T[i] >= 0.001 - M * (1 - Y[i]), name=f"Y_activate_{i}")
        # model.addConstr(SWT[i] - T[i] <= M * Y[i], name=f"Y_deactivate_{i}")
        # model.addConstr(SWT[i] >= T[i] - M * Y[i], name=f"Y_deactivate_{i}")
        # model.addConstr(SWT[i] <= T[i] + M * (1-Y[i]), name=f"Y_deactivate_{i}")
        model.addConstr(T[i] >= SWT[i] - M * Y[i], name=f"Y_activate_{i}")
        model.addConstr(T[i] <= SWT[i] + M * (1-Y[i]), name=f"Y_deactivate_{i}")
    
    # Constraint 5: Zi
    for i in surgeries:
        model.addConstr(Z[i] >= gp.quicksum(X[i, j, k] for j in days for k in rooms), name=f"Scheduled_{i}")
        model.addConstr(Z[i] <= gp.quicksum(X[i, j, k] for j in days for k in rooms), name=f"Link_Z_to_X_{i}")


    # Define the penalty term for surgeries not scheduled at least once
    penalty_unscheduled = gp.quicksum((1 - gp.quicksum(X[i, j, k] for j in days for k in rooms)) for i in surgeries)

    # Define the penalty term for surgeries exceeding target wait time
    penalty_term = gp.quicksum((SWT[i] - T[i]) * Y[i] for i in surgeries)

    # Define the cost term for each scheduled surgery
    cost_term = gp.quicksum(SWT[i] for i in surgeries)

    # Combine the penalty term and cost term to form the objective function
    model.setObjective(penalty_term + (beta * cost_term) + delta * penalty_unscheduled, GRB.MINIMIZE)

    # Optimize the model
    model.optimize()

    # Check if the optimization was successful
    if model.status == GRB.OPTIMAL:
        # Extract the schedule results
        schedule_results = []
        # Extract the schedule results and create DataFrame
        schedule_results = [{'Surgery': i, 'Day': j, 'Room': k} for i in surgeries for j in days for k in rooms if X[i, j, k].x > 0.5]
        schedule_results_df = pd.DataFrame(schedule_results)

        # Append or create new schedule DataFrame
        if schedule_df is not None:
            schedule = pd.concat([schedule_df, schedule_results_df], ignore_index=True)
        else:
            schedule = schedule_results_df

        # Extract the scheduled wait times
        scheduled_wait_times = {i: SWT[i].x for i in surgeries}

        # Update the capacity information dataframe
        updated_capacity_info_df = capacity_info_df.copy()
        for result in schedule_results:
            updated_capacity_info_df.loc[(updated_capacity_info_df['day'] == result['Day']) & (updated_capacity_info_df['room'] == result['Room']), 'capacity'] -= D[result['Surgery']]

        # Update the surgical portfolio dataframe
        updated_surgical_portfolio_df = surgical_info_df.copy()
        for result in schedule_results:
            updated_surgical_portfolio_df = updated_surgical_portfolio_df[updated_surgical_portfolio_df['id'] != result['Surgery']]
        # updated_surgical_portfolio_df['days_waited'] += 1

        #print vars:
        penalty_values = []
        for i in surgeries:
            swt_value = SWT[i].x
            target_wait_time = T[i]
            binary_indicator = Y[i].x
            scheduled = Z[i].x
            penalty_contribution = (swt_value - target_wait_time) * binary_indicator

            penalty_values.append({'Scheduled':scheduled,'Surgery': i, 'SWT': swt_value, 'Target Wait Time': target_wait_time, 'Y Indicator': binary_indicator, 'Penalty Contribution': penalty_contribution})

            print(f"Scheduled = {scheduled}, Surgery {i}: SWT = {swt_value}, Target Wait Time = {target_wait_time}, Y = {binary_indicator}, Penalty Contribution = {penalty_contribution}")

        return schedule, scheduled_wait_times, updated_capacity_info_df, updated_surgical_portfolio_df

    else:
        print("No feasible solution found.")
        return None, None, None, None
    
    


In [282]:

caps = pd.read_csv('OR_caps.csv')
surgicalPortfolio = pd.read_csv('surgeries_data.csv')
current_day = 1

schedule,wt,updated_caps,updated_surgical_portfolio = rolling_horizon_scheduler(surgicalPortfolio,caps, current_day)

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))



CPU model: 13th Gen Intel(R) Core(TM) i7-1355U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 185 rows, 240 columns and 960 nonzeros
Model fingerprint: 0x26c62cee
Model has 30 quadratic objective terms
Variable types: 30 continuous, 210 integer (210 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [1e-01, 1e+11]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+05]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 3.000000e+12
Presolve removed 90 rows and 50 columns
Presolve time: 0.00s
Presolved: 115 rows, 210 columns, 700 nonzeros
Variable types: 0 continuous, 210 integer (170 binary)
Found heuristic solution: objective 1.600000e+12

Root relaxation: objective 1.000000e+12, 160 iterations, 0.00 seconds (0.00 work units)

 

In [283]:
schedule.sort_values(['Day'])

,Surgery,Day,Room
1,S2_Tonsillectomy,1,0
11,S12_Tonsillectomy,1,0
7,S8_Tonsillectomy,1,0
5,S6_Tonsillectomy,1,0
0,S1_Tonsillectomy,2,0
8,S9_Tonsillectomy,2,0
9,S10_Tonsillectomy,2,0
6,S7_Tonsillectomy,2,0
4,S5_Tonsillectomy,3,0
2,S3_Tonsillectomy,3,0


In [284]:
updated_caps

,Unnamed: 0,day,room,capacity
0,0,1,0,0
1,1,2,0,0
2,2,3,0,0
3,3,4,0,0
4,4,5,0,0
5,5,6,0,4
6,6,7,0,4
7,7,8,0,4
8,8,9,0,4
9,9,10,0,4


In [285]:
updated_surgical_portfolio

,id,type,duration,ideal_waiting_time,days_waited
20,S21_ACL Reconstruction,ACL Reconstruction,1,10,0
21,S22_ACL Reconstruction,ACL Reconstruction,1,10,0
22,S23_ACL Reconstruction,ACL Reconstruction,1,10,0
23,S24_ACL Reconstruction,ACL Reconstruction,1,10,0
24,S25_ACL Reconstruction,ACL Reconstruction,1,10,0
25,S26_ACL Reconstruction,ACL Reconstruction,1,10,0
26,S27_ACL Reconstruction,ACL Reconstruction,1,10,0
27,S28_ACL Reconstruction,ACL Reconstruction,1,10,0
28,S29_ACL Reconstruction,ACL Reconstruction,1,10,0
29,S30_ACL Reconstruction,ACL Reconstruction,1,10,0


In [287]:
current_day += 1
schedule,wt,updated_caps,updated_surgical_portfolio = rolling_horizon_scheduler(updated_surgical_portfolio,updated_caps, current_day, schedule_df = schedule)
schedule.sort_values(['Day'])


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-1355U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 41 rows, 48 columns and 192 nonzeros
Model fingerprint: 0xfe53946e
Model has 6 quadratic objective terms
Variable types: 6 continuous, 42 integer (42 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [1e-01, 1e+11]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+05]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 6.000000e+11
Presolve removed 41 rows and 48 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

So

,Surgery,Day,Room
1,S2_Tonsillectomy,1,0
5,S6_Tonsillectomy,1,0
7,S8_Tonsillectomy,1,0
11,S12_Tonsillectomy,1,0
0,S1_Tonsillectomy,2,0
6,S7_Tonsillectomy,2,0
8,S9_Tonsillectomy,2,0
9,S10_Tonsillectomy,2,0
2,S3_Tonsillectomy,3,0
19,S20_Tonsillectomy,3,0
